# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

---

# Carga de datos

In [14]:
features = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
targets = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features.merge(targets, left_index=True, right_index=True)

---

# Preparación del set de datos para XGBoost

In [64]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [65]:
xg_reg = xgb.XGBRegressor(colsample_bytree = 0.7, learning_rate = 1,
                n_estimators = 100, n_jobs=6, max_depth=7, objective="reg:linear")

In [66]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=6, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [67]:
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 93545.206149


---

# Pruebas

In [75]:
features1 = pd.read_pickle("../../features/auctions_train_ventana_1.pkl")
targets1 = pd.read_pickle("../../targets/targets_ventana_1_auctions.pkl")
train = features1.merge(targets1, left_index=True, right_index=True)

features2 = pd.read_pickle("../../features/auctions_train_ventana_2.pkl")
targets2 = pd.read_pickle("../../targets/targets_ventana_2_auctions.pkl")
train.append(features2.merge(targets2, left_index=True, right_index=True))

X, y = train.iloc[:,:-1], train.iloc[:,-1]
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

param = {'max_depth':5,
         'eta':0.1,
         'silent':1,
         'objective':'reg:linear',
         'eval_metric' : 'rmse',
         'obj' : 'reg:squarederror',
         'colsample_bytree' : 0.7,
         'min_child_weight' : 0.5,
         'random_state' : 0,
         'reg_alpha' : 0,
         'reg_lambda' : 1,
         'scale_pos_weight' : 1,
         'learning_rate' : 0.01}


num_round = 5000
stopping = 20
bst = xgb.train(param, dtrain, num_round, evals=[(dtrain, "train"), (dtest, "test")], early_stopping_rounds=stopping)
# make prediction
#preds = bst.predict(dtest)

[0]	train-rmse:107883	test-rmse:104289
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:107275	test-rmse:103706
[2]	train-rmse:106701	test-rmse:103159
[3]	train-rmse:106110	test-rmse:102593
[4]	train-rmse:105546	test-rmse:102054
[5]	train-rmse:104972	test-rmse:101506
[6]	train-rmse:104421	test-rmse:100979
[7]	train-rmse:103863	test-rmse:100447
[8]	train-rmse:103321	test-rmse:99926.3
[9]	train-rmse:102779	test-rmse:99409
[10]	train-rmse:102256	test-rmse:98909.7
[11]	train-rmse:101728	test-rmse:98406.8
[12]	train-rmse:101222	test-rmse:97926.3
[13]	train-rmse:100724	test-rmse:97452.5
[14]	train-rmse:100220	test-rmse:96975.1
[15]	train-rmse:99722.2	test-rmse:96502.8
[16]	train-rmse:99231.6	test-rmse:96038.1
[17]	train-rmse:98759.7	test-rmse:95587.3
[18]	train-rmse:98280.6	test-rmse:95133.6
[19]	train-rmse:97808.8	test-rmse:94691.5
[20]	train-rmse:97344.9	test-rmse:94252.8
[21]	train

---

In [76]:
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor(metric='rmse', n_estimators=50, colsample_bytree=0.8)
#lgb_model = lgb.LGBMRegressor(boosting_type='dart', num_leaves=7, n_estimators=1000, metric='rmse',
#                               learning_rate=0.05, colsample_bytree=0.8)
lgb_model.fit(X_train, y_train)
preds = lgb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 69868.127718


In [77]:
preds_xgb = bst.predict(dtest)
preds_lgbm = lgb_model.predict(X_test)

In [79]:
p1 = 0.8
p2 = 0.2

preds = p1*preds_xgb + p2*preds_lgbm
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 69654.364553
